In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "영화에 대해서 3개의 이모티콘으로 표현해줘, 😊😊😊 "),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}."),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})['history']

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

invoke_chain("아바타")

content='🌿🌌🔥'
